In [56]:
import os

In [57]:
%pwd

'd:\\Repositories\\ML-geo'

In [58]:
os.chdir("d://Repositories/ML-geo/")

In [59]:
%pwd

'd:\\Repositories\\ML-geo'

In [60]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    

In [61]:
from src.XGboost_for_slopes.constants import*
from src.XGboost_for_slopes.utils.common import create_directories,read_yaml


In [68]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL= config.source_URL,
            local_data_file= config.local_data_file
            )
        return data_ingestion_config

In [69]:
import os
import urllib.request as request
from src.XGboost_for_slopes import logger
from src.XGboost_for_slopes.utils.common import get_size

In [70]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url= self.config.source_URL,
                filename= self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exist of size: {get_size(Path(self.config.local_data_file))}")

In [71]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
except Exception as e:
    raise e

[2023-12-06 19:14:17,298: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-06 19:14:17,300: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-06 19:14:17,302: INFO: common: created directory at: artifacts]
[2023-12-06 19:14:17,303: INFO: common: created directory at: artifacts/data_ingestion]
[2023-12-06 19:14:18,107: INFO: 2636294053: artifacts/data_ingestion/main_data.csv download! with following info: 
Server: GitHub.com
Date: Wed, 06 Dec 2023 22:14:18 GMT
Content-Type: text/html; charset=utf-8
Vary: X-PJAX, X-PJAX-Container, Turbo-Visit, Turbo-Frame, Accept-Encoding, Accept, X-Requested-With
ETag: W/"7f121da6a809430e52baf77b36757a1c"
Cache-Control: max-age=0, private, must-revalidate
Strict-Transport-Security: max-age=31536000; includeSubdomains; preload
X-Frame-Options: deny
X-Content-Type-Options: nosniff
X-XSS-Protection: 0
Referrer-Policy: no-referrer-when-downgrade
Content-Security-Policy: default-src 'none'; base-uri 'self'; child-src